# Trading Pipeline Demo

Demonstrates running the pipeline for a date range and a single day.

In [1]:
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [2]:
from trading_bot.agents.llm_roles import (
    MarketAnalystAgent,
    NewsSummarizerAgent,
    StrategyTypeAgent, 
    IndicatorSelectionAgent,
    StrategyPlannerAgent,
)

from trading_bot.coordinator import Coordinator
from trading_bot.pipeline import Pipeline
from trading_bot.storage import JSONStorage
from trading_bot.backtest import Backtester
from trading_bot.portfolio import Portfolio
import yfinance as yf
import pandas as pd 
from trading_bot.strategy import compose_strategy
from trading_bot.backtest import Backtester
from trading_bot.data.utils import save_json_payload
# Step 1: Get plans from your pipeline
import time
import trading_bot.strategies as _  # triggers the registrations
from trading_bot.strategies.registry import get_strategy_spec

coord = Coordinator([
    MarketAnalystAgent(),
    NewsSummarizerAgent(),
    StrategyTypeAgent(), 
    IndicatorSelectionAgent(),
    StrategyPlannerAgent(),
])

storage = JSONStorage()
backtester = Backtester()
portfolio = Portfolio()
pipe = Pipeline(coord, storage=storage, backtester=backtester, portfolio=portfolio)


# Test Agents

In [ ]:
single_result = pipe.run('TSLA', today='2025-02-04')
single_result['strategy'], single_result['files']


In [ ]:
single_result = pipe.run('TSLA', today='2025-03-04')
single_result['strategy'], single_result['files']

In [ ]:
range_result = pipe.run('TSLA', start_date='2025-02-03', end_date='2025-02-05')
range_result['files'], range_result['backtest']

# Print Yahoo historical data

In [ ]:
import yfinance as yf
import pandas as pd
end = pd.to_datetime("2025-02-04")
start = end - pd.Timedelta(days=7)
df = yf.download(
    "TSLA",
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    interval="1h",
    auto_adjust=True,
    progress=False,
)
df

In [ ]:
import yfinance as yf
import pandas as pd
end = pd.to_datetime("2025-03-04")
start = end - pd.Timedelta(days=7)
df = yf.download(
    "TSLA",
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    interval="1h",
    auto_adjust=True,
    progress=False,
    group_by="column",
)
df

# Test Backtester.py

In [ ]:
plans = []
for trade_date in pd.date_range("2025-03-02", "2025-03-04", freq="B"):
    date_str = trade_date.strftime("%Y-%m-%d")

    print(date_str)
    # Run your pipeline (this is where your screenshot saving already happens)
    agent_doc = pipe.run("TSLA", today=date_str)

    # Compose strategy plan
    plan = compose_strategy("TSLA", {"StrategyPlannerAgent": agent_doc}, strategy_date=trade_date)["plan"]
    # Save plan JSON in the same dated pattern as your other files
    save_json_payload("TSLA", {"plan": plan}, kind="plan", base_dir="data", day=date_str)

    # # Save plan JSON in the same dated pattern as your other files
    # save_json_payload("TSLA", {"plan": plan}, kind="strategy", day=date_str)

    # Keep collecting for the backtest
    plans.append({"symbol": "TSLA", "date": date_str, "plan": plan})

    time.sleep(5)  # adjust to stay under your rate limit

# Step 2: Backtest them
bt = Backtester()
results = bt.backtest_with_plans(plans)

# Step 3: Inspect
for r in results:
    print(r["date"], r.get("status"), r.get("final_value"))


In [ ]:
plan

In [ ]:
agent_doc

# Find which day trigger "Breakout" strategy

In [ ]:
import pandas as pd
import yfinance as yf
import trading_bot.strategies as _  # ensure specs register
from trading_bot.data.utils import get_previous_day_high
from trading_bot.strategies.registry import get_strategy_spec

symbol = "TSLA"
start = "2025-02-01"
end   = "2025-03-31"

# (Optional) pull extra history so the first few days have a "previous day"
hist_start = (pd.to_datetime(start) - pd.Timedelta(days=30)).strftime("%Y-%m-%d")

df = yf.download(symbol, start=hist_start, end=end, interval="1h",
                 progress=False, auto_adjust=True, multi_level_index=False,)
if df.empty:
    raise SystemExit("No data returned")
df = df.rename(columns=str.lower)
df.index.name = "datetime"

# ---- make index tz-naive to avoid 'tz-aware vs tz-naive' comparisons ----
if isinstance(df.index, pd.DatetimeIndex) and df.index.tz is not None:
    df = df.copy()
    df.index = df.index.tz_convert(None)

spec = get_strategy_spec("breakout")
params_default = {"volume_multiplier": 1.0, "min_body_pct": 0.3}  # loosened for testing

# Unique trading days present in df (tz-naive, sorted)
unique_days = (
    pd.DatetimeIndex(df.index.normalize())
    .tz_localize(None).unique().sort_values()
)

# Only scan within your original window
scan_days = [d for d in unique_days if (d.date() >= pd.to_datetime(start).date()
                                        and d.date() <= pd.to_datetime(end).date())]

hits = []
for d in scan_days[1:]:  # start from 2nd so a previous day exists
    try:
        pvh = get_previous_day_high(df, ref_date=d)  # util handles tz-naive
    except ValueError:
        continue  # no previous day in window; skip

    # slice this day (tz-naive normalize on both sides)
    day_bars = df[df.index.normalize() == d]
    if day_bars.empty:
        continue

    window = df[df.index <= day_bars.index.max()].tail(300)
    sig = spec.detector(window, previous_day_high=pvh, **params_default)

    # signals for this day only
    day_sig = sig.loc[sig.index.normalize() == d]
    # normalize to Series if it's a 1-col DataFrame
    if isinstance(day_sig, pd.DataFrame):
        day_sig = day_sig.iloc[:, 0]

    # now this won't raise the ambiguous truth error
    if not day_sig.empty and bool(day_sig.any()):
        ts = day_sig[day_sig].index[0]
        hits.append((d.date().isoformat(), ts.strftime("%Y-%m-%d %H:%M")))
        print(f"[HIT] {symbol} breakout on {d.date()} (first signal: {ts})")

print("\nSummary:")
print("\n".join(f"  {d} (first signal at {ts})" for d, ts in hits) or "No hits.")


## Test backtrader using the info from above

In [ ]:
# scripts/run_breakout_smoke.py
from trading_bot.strategy import compose_strategy
from trading_bot.backtest import Backtester

symbol = "TSLA"
trade_date = "2025-03-24"   # any recent weekday is fine

# Minimal planner dict (pretend from StrategyPlannerAgent)
planner_dict = {
    "strategy_type": "breakout",
    "summary": "Smoke test breakout",
    "rationale": "Force easier trigger via params",
    "setup": {
        "direction": "long",
        # keep stop/targets simple; backtester just needs numbers present
        "stop_loss": {"price": 1.0},
        "capital_risk_pct": 2.0,
        "targets": [{"price": 9_999, "size_pct": 100.0}],
        "trailing_stop": {"method": "ATR", "period": 14, "mult": 2.0},
        # ↓ Make detector easier to trigger for this smoke test
        "detector_params": {
            "volume_multiplier": 1.0,  # was 1.2
            "min_body_pct": 0.3        # was 0.5
        }
    },
}

plan = compose_strategy(symbol, {"StrategyPlannerAgent": planner_dict}, strategy_date=trade_date)["plan"]

bt = Backtester(starting_cash=100_000.0)
res = bt._run_single_backtest(symbol, trade_date, trade_date, plan)

print("Status:", res["status"])
print("Final Value:", res["final_value"])
print("Net Return:", f"{res['net_return']:.2%}")
print("Trades:", res["trade_analysis"].get("total"))
print("Applied Strategy:", res["strategy_applied"]["strategy_type"])


# Find which day trigger "Reversal" strategy

In [ ]:
import pandas as pd
import yfinance as yf
import trading_bot.strategies as _  # ensure specs register
from trading_bot.strategies.registry import get_strategy_spec

symbol = "TSLA"
start = "2025-02-01"
end   = "2025-03-31"

# (Optional) pull extra history so the first few days have indicator warmup
hist_start = (pd.to_datetime(start) - pd.Timedelta(days=30)).strftime("%Y-%m-%d")

df = yf.download(symbol, start=hist_start, end=end, interval="1h",
                 progress=False, auto_adjust=True, multi_level_index=False,)
if df.empty:
    raise SystemExit("No data returned")

df = df.rename(columns=str.lower)
df.index.name = "datetime"

# ---- make index tz-naive to avoid 'tz-aware vs tz-naive' comparisons ----
if isinstance(df.index, pd.DatetimeIndex) and df.index.tz is not None:
    df = df.copy()
    df.index = df.index.tz_convert(None)

# Build the trading-day list inside your [start, end] window
unique_days = (
    pd.DatetimeIndex(df.index.normalize())
    .tz_localize(None).unique().sort_values()
)
scan_days = [d for d in unique_days
             if (d.date() >= pd.to_datetime(start).date()
                 and d.date() <= pd.to_datetime(end).date())]

# Get the reversal spec (registered in trading_bot.strategies.__init__)
spec = get_strategy_spec("reversal")
if spec is None:
    raise SystemExit("Reversal strategy not registered. Ensure trading_bot/strategies/__init__.py registers it.")

hits = []
for d in scan_days[1:]:  # start from second day just to be safe with indicator warmup
    # Slice bars for this day
    day_bars = df[df.index.normalize() == d]
    if day_bars.empty:
        continue

    # Use a rolling window up to the last bar of the day (gives indicators enough history)
    window = df[df.index <= day_bars.index.max()].tail(300)

    # Reversal detector uses its own defaults; no breakout-only args
    sig = spec.detector(window)  # or spec.detector(window, params=overrides_dict)

    # Keep only today's signals
    day_sig = sig.loc[sig.index.normalize() == d]
    if isinstance(day_sig, pd.DataFrame):  # normalize to Series if needed
        day_sig = day_sig.iloc[:, 0]

    if not day_sig.empty and bool(day_sig.any()):
        ts = day_sig[day_sig].index[0]
        hits.append((d.date().isoformat(), ts.strftime("%Y-%m-%d %H:%M")))
        print(f"[HIT] {symbol} reversal on {d.date()} (first signal: {ts})")

print("\nSummary:")
print("\n".join(f"  {d} (first signal at {ts})" for d, ts in hits) or "No hits.")


[HIT] TSLA reversal on 2025-02-10 (first signal: 2025-02-10 20:30:00)
[HIT] TSLA reversal on 2025-02-11 (first signal: 2025-02-11 15:30:00)
[HIT] TSLA reversal on 2025-02-21 (first signal: 2025-02-21 18:30:00)
[HIT] TSLA reversal on 2025-02-24 (first signal: 2025-02-24 14:30:00)
[HIT] TSLA reversal on 2025-02-25 (first signal: 2025-02-25 14:30:00)
[HIT] TSLA reversal on 2025-03-10 (first signal: 2025-03-10 13:30:00)
[HIT] TSLA reversal on 2025-03-24 (first signal: 2025-03-24 13:30:00)

Summary:
  2025-02-10 (first signal at 2025-02-10 20:30)
  2025-02-11 (first signal at 2025-02-11 15:30)
  2025-02-21 (first signal at 2025-02-21 18:30)
  2025-02-24 (first signal at 2025-02-24 14:30)
  2025-02-25 (first signal at 2025-02-25 14:30)
  2025-03-10 (first signal at 2025-03-10 13:30)
  2025-03-24 (first signal at 2025-03-24 13:30)


In [ ]:
# scripts/run_reversal_smoke.py
import copy
from trading_bot.strategy import compose_strategy
from trading_bot.backtest import Backtester
from trading_bot.pipeline import Pipeline
import trading_bot.strategies as _  # ensure strategy specs register

symbol = "TSLA"
trade_date = "2025-02-24"  # one of your detected hit dates

# 1) Get actual agent outputs for that date
pipe = Pipeline(coord, storage=storage, backtester=backtester, portfolio=portfolio)
agent_doc = pipe.run(symbol, today=trade_date)  # returns full conversation summary per your pipeline

# 2) Pull the real planner dict (no hard-coding)
planner = agent_doc.get("StrategyPlannerAgent") or agent_doc  # depending on your pipeline’s shape

# ---- Optional: loosen detector params just for the smoke test ----
# This keeps everything else from the real output intact.
planner_smoke = copy.deepcopy(planner)
planner_smoke.setdefault("setup", {})
planner_smoke["setup"].setdefault("detector_params", {})
planner_smoke["setup"]["detector_params"].update({
    "rsi_window": 14,
    "bb_window": 20,
    "bb_dev": 2.0,
    "rsi_oversold": 40,   # easier to trigger than default 30
    "body_pct_min": 0.30, # easier to trigger than default 0.50
})

# DEFAULTS: Dict[str, Any] = {
#     "rsi_window": 14,
#     "bb_window": 20,
#     "bb_dev": 2.0,
#     "rsi_oversold": 30,
#     "rsi_overbought": 70,
#     "body_pct_min": 0.50,  # 50%
# }
# -----------------------------------------------------------------

# 3) Compose executable plan from the (optionally tweaked) real planner output
plan = compose_strategy(
    symbol,
    {"StrategyPlannerAgent": planner_smoke},
    strategy_date=trade_date
)["plan"]

# 4) Backtest just that day
bt = Backtester(starting_cash=100_000.0)
res = bt._run_single_backtest(symbol, trade_date, trade_date, plan)

print("Status:", res["status"])
print("Final Value:", res["final_value"])
print("Net Return:", f"{res['net_return']:.2%}")
print("Trades:", res["trade_analysis"].get("total"))
print("Applied Strategy:", res["strategy_applied"]["strategy_type"])
